In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pmdarima as pm
from functools import reduce
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pmdarima as pm
from functools import reduce
import statsmodels.api as sm
from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.stattools import adfuller
from scipy.optimize import minimize

In [ ]:
# function calculates the person days for each month. ie exposure - population times number of days in the month
def person_days(df):
    result = df.copy()
    for col in df.columns:
        mon = col[-2:]
        
        # don't multiply column which are relevant data 
        if len(col) < 5:
            result[col] = df[col]
            #print("Col " + col + " not multiplied")
            
            # September, April, June and November by 30
        elif mon in ['09','04','06','11']:
            result[col] = df[col]*30
            #print("Col " + col + " Multiplied by 30")
        elif mon in ['01','03','05','07','08','10','12']:
            result[col] = df[col]*31
            #print("Col " + col + " Multiplied by 31")
            
            # February leap years by 29
        elif mon == '02' and col[-5:-3] in ['16','20']:
            result[col] = df[col]*29
            #print("Col " + col + " Multiplied by 29")       
            
            # non-leap year february by 28
        elif mon == '02' and col[-5:-3] not in ['16','20']:
            result[col] = df[col]*28
            #print("Col " + col + " Multiplied by 28")
    return result

# undoes the previous function - divides by number of days in each month
def undo_person_days(df):
    result = df.copy()
    for col in df.columns:
        mon = col[-2:]
        
        # don't multiply column which are relevant data 
        if len(col) < 5:
            result[col] = df[col]
            #print("Col " + col + " not multiplied")
            
            # September, April, June and November by 30
        elif mon in ['09','04','06','11']:
            result[col] = df[col]/30
            #print("Col " + col + " Multiplied by 30")
        elif mon in ['01','03','05','07','08','10','12']:
            result[col] = df[col]/31
            #print("Col " + col + " Multiplied by 31")
            
            # February leap years by 29
        elif mon == '02' and col[-5:-3] in ['16','20']:
            result[col] = df[col]/29
            #print("Col " + col + " Multiplied by 29")       
            
            # non-leap year february by 28
        elif mon == '02' and col[-5:-3] not in ['16','20']:
            result[col] = df[col]/28
            #print("Col " + col + " Multiplied by 28")
    return result

In [6]:
def objective(k_t,a_x, b_x,D, pop, deaths,terms): # input K_t is a flat nx1 vector 
    k_t = k_t.reshape((terms,pop.shape[1]))
    exp_term = np.multiply(np.exp(a_x + b_x@np.diag(D)[:terms,:terms]@k_t),pop)
    return np.sum((deaths - exp_term)**2)

def regional_objective(k_t,b_x,K_t,B_x,a_xi,D_country,D,region_pop,region_death,regional_terms):
    k_t = k_t.reshape((regional_terms,region_pop.shape[1]))
    exp_term = np.multiply(np.exp(a_xi + B_x@np.diag(D_country)[:K_t.shape[0],:K_t.shape[0]]@K_t + b_x@np.diag(D)[:regional_terms,:regional_terms]@k_t),region_pop)
    return np.sum((region_death - exp_term)**2)



In [ ]:
def lag_exogenous(exog_table,numlags=12):
    factor_names = exog_table.columns
    numlags = 12
    for factor in factor_names:
        for l in range(numlags):
            name = factor + "_lag" + str(l+1)
            exog_table[name] = exog_table[factor].shift(l+1).bfill()
    return exog_table

In [ ]:
def ma_smoothing(df, window_size=3):
    # Function to apply rolling mean
    def apply_rolling(series):
        return series.rolling(window=window_size, center=True, min_periods=1).mean()

    # Apply the smoothing function to each row of the DataFrame
    smoothed_df = df.apply(apply_rolling, axis=1)
    return smoothed_df

In [6]:
def pop_adjust(df, apply_moving_average=False, window_size=12):
    """
    Adjust the population data for all age groups using proportionate allocation.
    
    Parameters:
    df (DataFrame): The dataframe containing population data with age groups as rows and months as columns.
    apply_moving_average (bool): Whether to apply moving average smoothing. Default is False.
    window_size (int): The window size for moving average. Default is 12.
    
    Returns:
    DataFrame: The adjusted population data for all age groups.
    """
    adjusted_df = df.copy()
    
    for index, row in df.iterrows():
        # Extract the specified age group data
        age_group_data = row.values

        # Initialize adjusted data for the specified age group
        adjusted_data = age_group_data.copy()

        # Calculate the yearly changes and distribute proportionally
        for i in range(1, len(age_group_data)//12):
            start_index = i * 12
            end_index = start_index + 12
            change = age_group_data[start_index] - age_group_data[start_index - 1]
            proportional_change = change / 12

            for j in range(12):
                adjusted_data[start_index + j] -= proportional_change * (j + 1)

        # Apply moving average if requested
        if apply_moving_average:
            adjusted_data = np.convolve(adjusted_data, np.ones(window_size)/window_size, mode='valid')
            # Adjust the length of the data to match the original
            adjusted_data = np.concatenate((adjusted_data, np.full(len(age_group_data) - len(adjusted_data), np.nan)))
        
        # Update the dataframe with the adjusted data
        adjusted_df.loc[index, :] = adjusted_data
    
    return adjusted_df


In [7]:
def full_model(country_data,regional_data,
               common_terms = 1,regional_terms = 1,
               prediction_period = 36,train_period = 84,
               exogenous_train = None,exogenous_test = None,
              country_deaths = None, country_pop = None,
              region_deaths = None, region_pop = None,re_est = False,
              jump_off = False,
              smooth = False):
    import numpy as np
    import matplotlib.pyplot as plt
    import pandas as pd
    import pmdarima as pm
    from functools import reduce
    from scipy.optimize import minimize
    
    if smooth:
        regional_data = ma_smoothing(regional_data)
        country_data = ma_smoothing(country_data)
    
    regional_logmort = regional_data.iloc[:,:train_period]
    country_logmort = country_data.iloc[:,:train_period]
    regional_test = regional_data.iloc[:,train_period:]
    country_test = country_data.iloc[:,train_period:]
        

    if country_data.shape != regional_data.shape:
        return " Error Country and Regional table shapes do not match"

     ### LC for country level 
    a_x = country_logmort.mean(axis = 1) #country_logmort.mean(axis = 1) # average over time for each age group
    A_xt =  country_logmort.sub(a_x,axis = 0) #logmort - a_x
    a_x = np.reshape(a_x,(len(a_x),1))

    U,D_country,V_t = np.linalg.svd(A_xt,full_matrices = False) # SVD of A_xt (should it be only logmort as in Lee-Li model?)

    B_xs = np.zeros((country_logmort.shape[0],common_terms)) # columns are B_x terms
    K_ts = np.zeros((common_terms,country_logmort.shape[1])) # rows are K_t terms 
    country_re_est = np.zeros(country_logmort.shape)
    for i in range(common_terms):
        b_x = U[:,i]
        b_x = b_x/(np.sqrt(np.sum(b_x**2))) # adjusting so that b_x^2 sums to 1 (works better why?)
        b_x = np.reshape(b_x,(len(b_x),))
        B_xs[:,i] = b_x
        b_x = np.reshape(b_x,(len(b_x),1))

        k_t = V_t[i,:]
        p_value = sm.stats.acorr_ljungbox(k_t, lags=[len(k_t) - 1],return_df = True).iloc[0,1]
        if p_value > 0.05:
            print("Common k_t"+str(i)+ " is white noise with LB p_value "+ str(p_value))
        k_t = k_t - np.mean(k_t) # adjusting so that k_t sums to 0
        k_t = np.reshape(k_t,(1,len(k_t)))
        K_ts[i,:] = k_t
        country_re_est = country_re_est + b_x@k_t
        country_re_est = B_xs@np.diag(D_country)[:common_terms,:common_terms]@K_ts
            
        

        ### LC for regional level
    regional_logmort_adj = regional_logmort - country_re_est
    a_xi = regional_logmort_adj.mean(axis = 1) # average over time for each age group
    if jump_off:
        a_xi = np.array(regional_data.iloc[:,train_period])
        a_xi = a_xi.reshape(regional_data.shape[0],1) 
    else:
        a_xi = np.reshape(a_xi, (len(a_xi), 1))
    
    A_xt_reg = regional_logmort_adj.sub(a_xi,axis = 1) #logmort - a_x
    #a_xi = np.reshape(a_x,(len(a_x),1))

    U,D,V_t = np.linalg.svd(A_xt_reg,full_matrices = False) # SVD of A_xt

    b_xs = np.zeros((regional_logmort.shape[0],regional_terms)) # columns are b_x terms
    k_ts = np.zeros((regional_terms,regional_logmort.shape[1])) # rows are k_t terms
    #regional_re_est = np.zeros(regional_logmort.shape)
    for i in range(regional_terms):
        b_x = U[:,i]
        b_x = b_x/(np.sqrt(np.sum(b_x**2))) # adjusting so that b_x^2 sums to 1 (works better why?)
        b_x = np.reshape(b_x,(len(b_x),))
        b_xs[:,i] = b_x
        b_x = np.reshape(b_x,(len(b_x),1))

        k_t = V_t[i,:]
        p_value = sm.stats.acorr_ljungbox(k_t, lags=[len(k_t) - 1],return_df = True).iloc[0,1]
        print("p_value k_t" +str(i)+" before re-estimation: " +str(p_value)+" \n")
        if p_value > 0.05:
            print("Regional k_t"+str(i)+ " is white noise with LB p_value "+ str(p_value)+ " \n")
        k_t = k_t - np.mean(k_t) # adjusting so that k_t sums to 0
        k_t = np.reshape(k_t,(1,len(k_t)))
        k_ts[i,:] = k_t
            
        #regional_re_est = regional_re_est + b_x@k_t
   
    # re-estimation of k_t terms
    res = 0
    if re_est:
        K_ts_reest = K_ts.flatten() #flatten as minimize only takes 1d array of parameters
        #print(objective(K_ts_reest,np.zeros((country_pop.shape[0],1)), B_xs,D_country, country_pop, country_deaths,common_terms))
        result = minimize(objective, K_ts_reest, args=(a_xi, B_xs,D_country, country_pop, country_deaths,common_terms), method='BFGS') #np.zeros((country_pop.shape[0],1))
        #print(objective(result.x,np.zeros((country_pop.shape[0],1)), B_xs,D_country, country_pop, country_deaths,common_terms))
        K_ts = result.x.reshape(((common_terms,country_pop.shape[1])))
        
        k_ts_reest = k_ts.flatten()
        print("Objecive before minimisation " +str(regional_objective(k_ts_reest,b_xs,K_ts,B_xs,a_xi,D_country,D,region_pop,region_deaths,regional_terms)))
        result = minimize(regional_objective,k_ts_reest,args = (b_xs,K_ts,B_xs,a_xi,D_country,D,region_pop,region_deaths,regional_terms),method = 'BFGS')
        print("Objective after minimisation " + str(regional_objective(result.x,b_xs,K_ts,B_xs,a_xi,D_country,D,region_pop,region_deaths,regional_terms)))
        k_ts = result.x.reshape(((regional_terms,region_pop.shape[1])))
        
    #regional_re_est = regional_re_est + a_xi
    regional_re_est = a_xi + B_xs@np.diag(D_country)[:common_terms,:common_terms]@K_ts + b_xs@np.diag(D)[:regional_terms,:regional_terms]@k_ts 
    #a_xi_adjustment = np.subtract(np.array(regional_logmort),regional_re_est)
    #a_xi = a_xi + np.mean(a_xi_adjustment,axis = 1).transpose()
    #regional_re_est = a_xi + B_xs@np.diag(D_country)[:common_terms,:common_terms]@K_ts + b_xs@np.diag(D)[:regional_terms,:regional_terms]@k_ts
    a_xi = np.reshape(np.mean(np.subtract(np.array(regional_logmort),regional_re_est),axis = 1),(regional_logmort.shape[0],1)) +a_xi
    regional_re_est = a_xi + B_xs@np.diag(D_country)[:common_terms,:common_terms]@K_ts + b_xs@np.diag(D)[:regional_terms,:regional_terms]@k_ts

        # model country terms    
    K_t_models = {}
    K_t_preds = np.zeros((common_terms,train_period + prediction_period))
    K_t_preds_upconf = np.zeros((common_terms,train_period + prediction_period))
    K_t_preds_lowconf = np.zeros((common_terms,train_period + prediction_period)) #country_logmort.shape[0]
    for i in range(common_terms):
        K_t_model = pm.auto_arima(K_ts[i,:].flatten(),
                               start_p = 0,
                               start_q = 0,
                               max_p = 15,
                               max_q = 15,
                               max_d = 4,
                               m = 12,
                               seasonal = True,
                               suppress_warnings=True,
                               error_action='ignore',with_intercept = True)

        K_t_models[f'model_{i}'] = K_t_model
        K_t_model_preds,K_t_model_confint = K_t_model.predict(n_periods = prediction_period,return_conf_int = True)
        K_t_preds[i,:] = np.append(K_ts[i,:],np.array(K_t_model_preds).reshape((1,len(K_t_model_preds))))
        K_t_preds_upconf[i,:] = np.append(K_ts[i,:],np.array(K_t_model_confint[:,0]).reshape((1,len(K_t_model_confint[:,0]))))
        K_t_preds_lowconf[i,:] = np.append(K_ts[i,:],np.array(K_t_model_confint[:,1]).reshape((1,len(K_t_model_confint[:,1]))))
    

        # models for regional terms 
    k_t_models = {}
    k_t_preds = np.zeros((regional_terms,train_period + prediction_period))
    k_t_preds_upconf = np.zeros((regional_terms,train_period + prediction_period))
    k_t_preds_lowconf = np.zeros((regional_terms,train_period + prediction_period)) #regional_logmort.shape[0]
    for i in range(regional_terms):
        k_t_model = pm.auto_arima(k_ts[i,:].flatten(),
                               exogenous_train,
                               start_p = 0,
                               start_q = 0,
                               max_p = 15,
                               max_q = 15,
                               max_d = 4,
                               m = 12,
                               seasonal = True,
                               suppress_warnings=True,
                               error_action='ignore',with_intercept = True)

        k_t_models[f'model_{i}'] = k_t_model
        k_t_model_preds,k_t_model_confint = k_t_model.predict(n_periods = prediction_period,X = exogenous_test,return_conf_int = True)
        k_t_preds[i,:] = np.append(k_ts[i,:],np.array(k_t_model_preds).reshape((1,len(k_t_model_preds))))
        k_t_preds_upconf[i,:] = np.append(k_ts[i,:],np.array(k_t_model_confint[:,0]).reshape((1,len(k_t_model_confint[:,0]))))
        k_t_preds_lowconf[i,:] = np.append(k_ts[i,:],np.array(k_t_model_confint[:,1]).reshape((1,len(k_t_model_confint[:,1]))))
            
            
    if jump_off:
        a_xi = np.array(regional_data.iloc[:,train_period])
        a_xi = a_xi.reshape(regional_data.shape[0],1)
        
    
    regional_predictions = a_xi + B_xs@np.diag(D_country)[:common_terms,:common_terms]@K_t_preds + b_xs@np.diag(D)[:regional_terms,:regional_terms]@k_t_preds  
    regional_predictions_upconf = a_xi + B_xs@np.diag(D_country)[:common_terms,:common_terms]@K_t_preds_upconf + b_xs@np.diag(D)[:regional_terms,:regional_terms]@k_t_preds_upconf 
    regional_predictions_lowconf = a_xi + B_xs@np.diag(D_country)[:common_terms,:common_terms]@K_t_preds_lowconf + b_xs@np.diag(D)[:regional_terms,:regional_terms]@k_t_preds_lowconf
    
    
    regional_preds = pd.DataFrame(regional_predictions)
    regional_preds.index = country_data.index
    regional_preds.columns = country_data.columns
    errors = MAPE(regional_preds.iloc[:,train_period:],country_data.iloc[:,train_period:])
    model_error = np.sum(errors)
    
    for i in range(regional_terms):
        p_value = sm.stats.acorr_ljungbox(k_ts[i,:], lags=[len(k_ts[i,:]) - 1],return_df = True).iloc[0,1]
        print("p_value k_t" +str(i)+" after re-estimation: " +str(p_value)+ " \n")
        if p_value > 0.05:
                print("Regional k_t"+str(i)+ " is white noise with LB p_value "+ str(p_value)+" \n")
        

    return a_xi,B_xs,K_ts,b_xs,k_ts,country_re_est,regional_re_est,K_t_models,k_t_models,K_t_preds,k_t_preds,regional_predictions,regional_predictions_upconf,regional_predictions_lowconf,model_error,res,D_country,D
    

    

In [3]:
# Mean Absolute Percentage Error
def MAPE(predictions,test_set):
    errors = test_set.subtract(predictions)
    results = []
    for i in range(0,test_set.shape[0]): 
        results = np.append(results,np.mean(np.abs(errors.iloc[i,:].divide(test_set.iloc[i,:])))*100)
    return results

# symmetric Mean Absolute Percentage Error
def sMAPE(predictions,test_set):
    errors = test_set.subtract(predictions)
    results = []
    for i in range(0,test_set.shape[0]):
        sum_pred_act = np.abs(test_set.iloc[i,:]) + np.abs(predictions.iloc[i,:])
        err = 2*np.abs(errors.iloc[i,:])
        results = np.append(results,np.mean(err.divide(sum_pred_act))*100)
    return results
    
# Root Mean Squared Error
def RMSE(predictions,test_set):
    errors = test_set.subtract(predictions)
    results = []
    for i in range(0,test_set.shape[0]):
        results = np.append(results,np.sqrt(np.mean(errors.iloc[i,:]**2)))
    return results


In [1]:
def prediction_plots(prediction_matrix,
          prediction_upconf_matrix,
          prediction_lowconf_matrix,
          test_period,
          model_data,
          suptitle = "Title",
          n=6,
          tick_rotation = 45,
          figure_size = (13,20)
         ):
    plt.figure()
    fig,ax = plt.subplots(model_data.shape[0],1,figsize = figure_size)
    fig.supylabel("Log-Mortality Rate")
    
    for i in range(model_data.shape[0]):
        ax[i].plot(prediction_matrix.iloc[i,:],label = "Estimates")
        ax[i].plot(model_data.loc[model_data.index[i]],label = "True Series")
        ax[i].set_title(model_data.index[i])
        ax[i].set_xticks(ax[i].get_xticks()[::n])
        ax[i].tick_params(rotation = tick_rotation)
        ax[i].axvspan(len(model_data.loc[model_data.index[0]]) - test_period, len(model_data.loc[model_data.index[0]]) , color='green', alpha=0.25)
        ax[i].plot(prediction_upconf_matrix.iloc[i,-test_period:],color = "red",linestyle = "dashed",label = "95% Confidence Interval")
        ax[i].plot(prediction_lowconf_matrix.iloc[i,-test_period:],color = "red",linestyle = "dashed")
        ax[i].legend()
        
    fig.suptitle(suptitle, y=1.02)  # Adjust the y position of the suptitle
    fig.tight_layout()
    fig.subplots_adjust(top=0.99) 
    plt.show();

In [8]:
def exog_variables(temp_factors,region,train_period = 60,test_period = 24):
    av_temp = np.array(temp_factors.loc[temp_factors['name'] == region].loc[:,"monthly_avgtemp"])
    av_temp_train = av_temp[:train_period]
    av_temp_test = av_temp[train_period:train_period+test_period]

    temp_below = np.array(temp_factors.loc[temp_factors['name'] == region].loc[:,"days_below_7"])
    temp_below_train = temp_below[:train_period]
    temp_below_test = temp_below[train_period:train_period+test_period]

    av_humid = np.array(temp_factors.loc[temp_factors['name'] == region].loc[:,"monthly_avghumidity"])
    av_humid_train = av_humid[:train_period]
    av_humid_test = av_humid[train_period:train_period+test_period]
    
    temp_above = np.array(temp_factors.loc[temp_factors['name'] == region].loc[:,"days_above_20"])
    temp_above_train = temp_above[:train_period]
    temp_above_test = temp_above[train_period:train_period+test_period]

    exog_train = np.zeros((len(av_temp_train),4))
    exog_train[:,0] = av_temp_train
    exog_train[:,1] = av_humid_train
    exog_train[:,2] = temp_below_train
    exog_train[:,3] = temp_above_train
    exog_train = pd.DataFrame(exog_train)
    exog_train.columns = ["av_temp","av_humid","temp_below","temp_above"]

    exog_test = np.zeros((len(av_temp_test),4))
    exog_test[:,0] = av_temp_test
    exog_test[:,1] = av_humid_test
    exog_test[:,2] = temp_below_test
    exog_test[:,3] = temp_above_test
    exog_test = pd.DataFrame(exog_test)
    exog_test.columns = ["av_temp","av_humid","temp_below","temp_above"]
    
    return exog_train,exog_test

In [9]:
##### experimental

def k_t_terms(country_data,regional_data, common_terms = 1,regional_terms = 1,prediction_period = 36,train_period = 84):
    import numpy as np
    import matplotlib.pyplot as plt
    import pandas as pd
    import pmdarima as pm
    from functools import reduce
    import statsmodels.api as sm
    from statsmodels.tsa.statespace.varmax import VARMAX
        
    
    regional_logmort = regional_data.iloc[:,:train_period]
    country_logmort = country_data.iloc[:,:train_period]
    regional_test = regional_data.iloc[:,train_period:]
    country_test = country_data.iloc[:,train_period:]
        

    if country_data.shape != regional_data.shape:
        return " Error Country and Regional table shapes do not match"

     ### LC for country level 
    a_x = country_logmort.mean(axis = 1) # average over time for each age group
    a_x = np.zeros(a_x.shape)
    A_xt = country_logmort.sub(a_x,axis = 0) #logmort - a_x
    a_x = np.reshape(a_x,(len(a_x),1))

    U,D_country,V_t = np.linalg.svd(A_xt,full_matrices = False) # SVD of A_xt (should it be only logmort as in Lee-Li model?)

    B_xs = np.zeros((country_logmort.shape[0],common_terms)) # columns are B_x terms
    K_ts = np.zeros((country_logmort.shape[1],common_terms)) # rows are K_t terms 
    country_re_est = np.zeros(country_logmort.shape)
    for i in range(common_terms):
        b_x = U[:,i]
        b_x = b_x/(np.sqrt(np.sum(b_x**2))) # adjusting so that b_x^2 sums to 1 (works better why?)
        b_x = np.reshape(b_x,(len(b_x),))
        B_xs[:,i] = b_x
        b_x = np.reshape(b_x,(len(b_x),1))

        k_t = V_t[i,:]
        k_t = k_t - np.mean(k_t) # adjusting so that k_t sums to 0
        k_t = np.reshape(k_t,(len(k_t),))
        K_ts[:,i] = k_t
        #country_re_est = country_re_est + b_x@k_t
            
        

        ### LC for regional level
    regional_logmort_adj = regional_logmort - country_re_est
    a_xi = regional_logmort_adj.mean(axis = 1) # average over time for each age group
    A_xt_reg = regional_logmort_adj.sub(a_xi,axis = 0) #logmort - a_x
    a_xi = np.reshape(a_x,(len(a_x),1))

    U,D,V_t = np.linalg.svd(A_xt_reg,full_matrices = False) # SVD of A_xt

    b_xs = np.zeros((regional_logmort.shape[0],regional_terms)) # columns are b_x terms
    k_ts = np.zeros((regional_logmort.shape[1],regional_terms)) # rows are k_t terms
    regional_re_est = np.zeros(regional_logmort.shape)
    for i in range(regional_terms):
        b_x = U[:,i]
        b_x = b_x/(np.sqrt(np.sum(b_x**2))) # adjusting so that b_x^2 sums to 1 (works better why?)
        b_x = np.reshape(b_x,(len(b_x),))
        b_xs[:,i] = b_x
        b_x = np.reshape(b_x,(len(b_x),1))

        k_t = V_t[i,:]
        k_t = k_t - np.mean(k_t) # adjusting so that k_t sums to 0
        k_t = np.reshape(k_t,(len(k_t),))
        k_ts[:,i] = k_t
            
        #regional_re_est = regional_re_est + b_x@k_t
    #regional_re_est = regional_re_est + a_xi

    all_k_t = pd.DataFrame(np.hstack((K_ts,k_ts)))
    

    return all_k_t, K_ts,k_ts,B_xs,b_xs,a_xi

    

In [1]:
##### experimental
def reverse_differencing(diff_series,initial_value):
    diff_inv = [initial_value]

    for i in range(len(diff_series)-2):
        diff_inv = np.append(diff_inv,diff_inv[i] + diff_series.iloc[i+2])

    diff_inv = pd.DataFrame(diff_inv)
    return diff_inv

def VARMAX_model(country_data,regional_data,
                 common_terms = 1,regional_terms = 1,
                 prediction_period = 36,train_period = 84,
                 AR = 2,MA = 2,iters = 100,
                 exogenous_train = None,exogenous_test = None,
                country_deaths = None, country_pop = None,
                region_deaths = None, region_pop = None,
                 re_est = False,smooth = False):
    import numpy as np
    import matplotlib.pyplot as plt
    import pandas as pd
    import pmdarima as pm
    from functools import reduce
    import statsmodels.api as sm
    from statsmodels.tsa.statespace.varmax import VARMAX
    from statsmodels.tsa.stattools import adfuller
    from scipy.optimize import minimize
    
    if smooth:
        regional_data = ma_smoothing(regional_data)
        country_data = ma_smoothing(country_data)
        
    
    regional_logmort = regional_data.iloc[:,:train_period]
    country_logmort = country_data.iloc[:,:train_period]
    regional_test = regional_data.iloc[:,train_period:]
    country_test = country_data.iloc[:,train_period:]
        

    if country_data.shape != regional_data.shape:
        return " Error Country and Regional table shapes do not match"

     ### LC for country level 
    a_x = country_logmort.mean(axis = 1) # average over time for each age group
    A_xt = country_logmort.sub(a_x,axis = 0) #logmort - a_x
    a_x = np.reshape(a_x,(len(a_x),1))

    U,D_country,V_t = np.linalg.svd(A_xt,full_matrices = False) # SVD of A_xt (should it be only logmort as in Lee-Li model?)

    B_xs = np.zeros((country_logmort.shape[0],common_terms)) # columns are B_x terms
    K_ts = np.zeros((country_logmort.shape[1],common_terms)) # rows are K_t terms 
    country_re_est = np.zeros(country_logmort.shape)
    for i in range(common_terms):
        b_x = U[:,i]
        b_x = b_x/(np.sqrt(np.sum(b_x**2))) # adjusting so that b_x^2 sums to 1 (works better why?)
        b_x = np.reshape(b_x,(len(b_x),))
        B_xs[:,i] = b_x
        b_x = np.reshape(b_x,(len(b_x),1))

        k_t = V_t[i,:]
        k_t = k_t - np.mean(k_t) # adjusting so that k_t sums to 0
        k_t = np.reshape(k_t,(len(k_t),))
        K_ts[:,i] = k_t
        #country_re_est = country_re_est + b_x@k_t
            
        

        ### LC for regional level
    regional_logmort_adj = regional_logmort - country_re_est
    a_xi = regional_logmort_adj.mean(axis = 1) # average over time for each age group
    A_xt_reg = regional_logmort_adj.sub(a_xi,axis = 0) #logmort - a_x
    a_xi = np.reshape(a_x,(len(a_x),1))

    U,D,V_t = np.linalg.svd(A_xt_reg,full_matrices = False) # SVD of A_xt

    b_xs = np.zeros((regional_logmort.shape[0],regional_terms)) # columns are b_x terms
    k_ts = np.zeros((regional_logmort.shape[1],regional_terms)) # rows are k_t terms
    regional_re_est = np.zeros(regional_logmort.shape)
    for i in range(regional_terms):
        b_x = U[:,i]
        b_x = b_x/(np.sqrt(np.sum(b_x**2))) # adjusting so that b_x^2 sums to 1 (works better why?)
        b_x = np.reshape(b_x,(len(b_x),))
        b_xs[:,i] = b_x
        b_x = np.reshape(b_x,(len(b_x),1))

        k_t = V_t[i,:]
        k_t = k_t - np.mean(k_t) # adjusting so that k_t sums to 0
        k_t = np.reshape(k_t,(len(k_t),))
        k_ts[:,i] = k_t
            
        #regional_re_est = regional_re_est + b_x@k_t
    #regional_re_est = regional_re_est + a_xi
    if re_est:
        K_ts_reest = K_ts.transpose()
        #print(K_ts_reest.shape)
        K_ts_reest = K_ts.flatten() #flatten as minimize only takes 1d array of parameters
        #print(objective(K_ts_reest,np.zeros((country_pop.shape[0],1)), B_xs,D_country, country_pop, country_deaths,common_terms))
        result = minimize(objective, K_ts_reest, args=(a_x, B_xs,D_country, country_pop, country_deaths,common_terms), method='BFGS') #np.zeros((country_pop.shape[0],1))
        #print(objective(result.x,np.zeros((country_pop.shape[0],1)), B_xs,D_country, country_pop, country_deaths,common_terms))
        K_ts = result.x.reshape(((common_terms,country_pop.shape[1])))
        K_ts = K_ts.transpose()
        
        k_ts_reest = k_ts.transpose()
        k_ts_reest = k_ts.flatten()
#         print("k_ts " +str(k_ts_reest.shape))
#         print("b_xs " + str(b_xs.shape))
#         print("K_ts " + str(K_ts.shape))
#         print("B_xs " + str(B_xs.shape))
#         print("a_xi " + str(a_xi.shape))
#         print("region_pop " + str(region_pop.shape))
#         print("region_deaths " + str(region_deaths.shape))
#         print("Objective before minimsation " + str(regional_objective(k_ts_reest,b_xs,K_ts.transpose(),B_xs,a_xi,region_pop,region_deaths,regional_terms)))
        result = minimize(regional_objective,k_ts_reest,args = (b_xs,K_ts.transpose(),B_xs,a_xi,D_country,D,region_pop,region_deaths,regional_terms),method = 'BFGS')
        #print("Objective before minimsation " + str(regional_objective(result.x,b_xs,K_ts.transpose(),B_xs,a_xi,region_pop,region_deaths,regional_terms)))
        k_ts = result.x.reshape(((regional_terms,region_pop.shape[1])))
        k_ts = k_ts.transpose()
    
        
    all_k_t = pd.DataFrame(np.hstack((K_ts,k_ts)))
    original = all_k_t
                                                                  
                                                        
    # dealing with non-stationarity                                                       
    differences = [] # number differences done for each time-series/column
    non_stat_cols = [] # returns index of non-stationary columns 
    for i in range(all_k_t.shape[1]):
        series = all_k_t.iloc[:,i]
        p_value = adfuller(series)[1]
        diff = 0
        if p_value > 0.05:
            non_stat_cols = np.append(non_stat_cols,i)
            #print("Non-Stationary k term : " + str(i))
            
        while p_value > 0.05:
            series = series.diff()
            series.iloc[0] = series.iloc[1]
            p_value = adfuller(series)[1]
            diff = diff + 1
            #print(diff)
            #print(p_value)
        differences = np.append(differences,diff)
        all_k_t.iloc[:,i] = series
        #print(differences)
        #print("Non_stat_cols" + str(non_stat_cols))
    
        
    
    model = VARMAX(all_k_t, order = (AR,MA),exog = exogenous_train,enforce_stationarity = False ,initialization = "approximate_diffuse")
    results = model.fit(maxiter = iters,disp = False)
    preds = results.get_forecast(steps = prediction_period,exog = exogenous_test)
    all_k_lowconf = preds.conf_int().iloc[:,:common_terms+regional_terms]
    all_k_upconf = preds.conf_int().iloc[:,common_terms+regional_terms:]
    
    all_k_preds = preds.predicted_mean
    all_k_preds = pd.concat([all_k_t,all_k_preds])

    all_k_lowconf.columns = all_k_preds.columns
    all_k_upconf.columns = all_k_preds.columns
    all_k_preds_lowconf = pd.concat([all_k_t,all_k_lowconf])
    all_k_preds_upconf = pd.concat([all_k_t,all_k_upconf])
    
    # reversing any differencing done 
    for i in non_stat_cols:
        pred_rev = reverse_differencing(all_k_preds.iloc[:,int(i)],original.iloc[0,int(i)])
        #pred_rev = pred_rev[1:]
        all_k_preds.iloc[:,int(i)] = pred_rev
        
        pred_lowconf_rev = reverse_differencing(all_k_preds_lowconf.iloc[:,int(i)],original.iloc[0,int(i)])
        #pred_lowconf_rev = pred_lowconf_rev[1:]
        all_k_preds_lowconf.iloc[:,int(i)] = pred_lowconf_rev
        
        pred_upconf_rev = reverse_differencing(all_k_preds_upconf.iloc[:,int(i)],[original.iloc[0,int(i)]])
        #pred_upconf_rev = pred_upconf_rev[1:]
        all_k_preds_upconf.iloc[:,int(i)] = pred_upconf_rev
        
    predictions = pd.DataFrame(B_xs@np.diag(D_country)[:common_terms,:common_terms]@np.array(all_k_preds.iloc[:,0:common_terms].transpose()) + b_xs@np.diag(D)[:regional_terms,:regional_terms]@np.array(all_k_preds.iloc[:,common_terms:].transpose()) + a_xi)
    predictions_lowconf = pd.DataFrame(B_xs@np.diag(D_country)[:common_terms,:common_terms]@np.array(all_k_preds_lowconf.iloc[:,0:common_terms].transpose()) + b_xs@np.diag(D)[:regional_terms,:regional_terms]@np.array(all_k_preds_lowconf.iloc[:,common_terms:].transpose()) + a_xi)
    predictions_upconf = pd.DataFrame(B_xs@np.diag(D_country)[:common_terms,:common_terms]@np.array(all_k_preds_upconf.iloc[:,0:common_terms].transpose()) + b_xs@np.diag(D)[:regional_terms,:regional_terms]@np.array(all_k_preds_upconf.iloc[:,common_terms:].transpose()) + a_xi)
    
    preds = pd.DataFrame(predictions)
    test_preds = preds.iloc[:,train_period:]
    test_preds.index = regional_test.index
    test_preds.columns = regional_test.columns
    

    test_errors = MAPE(test_preds,regional_test)

    aic = results.aic
    return all_k_t, K_ts,k_ts,B_xs,b_xs,a_xi,all_k_preds,predictions,predictions_lowconf,predictions_upconf,results,aic,test_errors,D_country,D

    

In [4]:
def LC_model(logmort_table,no_terms = 1,re_est = False,pop = None,death = None,prediction_period = 36):
    
    a_x = logmort_table.mean(axis = 1) # average over time for each age group
    A_xt = logmort_table.sub(a_x,axis = 0) #logmort - a_x
    a_x = np.reshape(a_x,(len(a_x),1))

    U,D,V_t = np.linalg.svd(A_xt,full_matrices = False) # SVD of A_xt (should it be only logmort as in Lee-Li model?)
   
    b_xs = np.zeros((logmort_table.shape[0],no_terms)) 
    k_ts = np.zeros((no_terms,logmort_table.shape[1]))  
    
    for i in range(no_terms):
        b_x = U[:,i]
        b_x = b_x/(np.sqrt(np.sum(b_x**2))) # adjusting so that b_x^2 sums to 1 (works better why?)
        #b_x = b_x/(np.sum(b_x))
        b_x = np.reshape(b_x,(len(b_x),))
        b_xs[:,i] = b_x
        b_x = np.reshape(b_x,(len(b_x),1))

        k_t = V_t[i,:]
        k_t = k_t - np.mean(k_t) # adjusting so that k_t sums to 0
        k_t = np.reshape(k_t,(len(k_t),))
        k_ts[i,:] = k_t
        
    if re_est:
        
        k_ts_reest = k_ts.flatten()
        print("Objecive before minimisation " +str(objective(k_ts_reest,a_xi,b_xs,D,pop,death,no_terms)))
        result = minimize(objective,k_ts_reest,args = (a_xi,b_xs,D,pop,death,no_terms),method = 'BFGS')
        print("Objective after minimisation " + str(objective(result.x,a_xi,b_xs,D,pop,death,no_terms)))
        k_ts = result.x.reshape(((no_terms,pop.shape[1])))
        
    
    re_est = pd.DataFrame(a_x + b_xs@np.diag(D)[:no_terms,:no_terms]@k_ts,columns = logmort_table.columns,index = logmort_table.index)

    return re_est,a_x,b_xs,k_ts,D
    

In [9]:

def distribute_pop(df):
    # Initialize the output DataFrame with zeros and the same index and columns as the input df
    output_df = pd.DataFrame(np.zeros(df.shape), index=df.index, columns=df.columns)

    # Populate the output DataFrame
    for row in range(df.shape[0]):
        for i in range(df.shape[1]):
            if i % 12 == 0:
                output_df.iloc[row, i] = df.iloc[row, i]
            else:
                # Calculate the incremental difference only if next_base_col is within bounds
                base_col = (i // 12) * 12
                next_base_col = base_col + 12
                if next_base_col < df.shape[1]:
                    increment = (df.iloc[row, next_base_col] - df.iloc[row, base_col]) / 12
                    # Add the appropriate multiple of the increment to the base column value
                    output_df.iloc[row, i] = df.iloc[row, base_col] + increment * (i % 12)
                else:
                    # If next_base_col is out of bounds, use the last valid increment
                    increment = (df.iloc[row, -1] - df.iloc[row, base_col]) / (df.shape[1] - base_col)
                    output_df.iloc[row, i] = df.iloc[row, base_col] + increment * (i % 12)

    return output_df

In [ ]:
def model_no_common(regional_data,regional_terms = 1,
               prediction_period = 36,train_period = 84,
               exogenous_train = None,exogenous_test = None,
               region_deaths = None, region_pop = None,
               re_est = False,smooth = False):
    import numpy as np
    import matplotlib.pyplot as plt
    import pandas as pd
    import pmdarima as pm
    from functools import reduce
    from scipy.optimize import minimize
    
    # applies a moving average smoothing to the data 
    if smooth:
        regional_data = ma_smoothing(regional_data)
    
    # create training and testing sets
    regional_logmort = regional_data.iloc[:,:train_period]    
    regional_test = regional_data.iloc[:,train_period:]
        
                  
        ### LC for regional level
    regional_logmort_adj = regional_logmort
    a_xi = regional_logmort_adj.mean(axis = 1) # average over time for each age group
    a_xi = np.reshape(a_xi,(len(a_xi),1))
    A_xt_reg = regional_logmort_adj.sub(a_xi,axis = 1) #logmort - a_x ,SVD applied to this (centering of matrix) 
    
    U,D,V_t = np.linalg.svd(A_xt_reg,full_matrices = False) # SVD of A_xt

    b_xs = np.zeros((regional_logmort.shape[0],regional_terms)) # columns are b_x terms
    k_ts = np.zeros((regional_terms,regional_logmort.shape[1])) # rows are k_t terms (right singular vectors)
    #regional_re_est = np.zeros(regional_logmort.shape)
    for i in range(regional_terms):
        b_x = U[:,i]
        b_x = b_x/(np.sqrt(np.sum(b_x**2))) # adjusting so that b_x^2 sums to 1 (works better why?)
        b_x = np.reshape(b_x,(len(b_x),))
        b_xs[:,i] = b_x
        b_x = np.reshape(b_x,(len(b_x),1))

        k_t = V_t[i,:]
        p_value = sm.stats.acorr_ljungbox(k_t, lags=[len(k_t) - 1],return_df = True).iloc[0,1]
        print("p_value k_t" +str(i)+" before re-estimation: " +str(p_value)+" \n")
        if p_value > 0.05:
            print("Regional k_t"+str(i)+ " is white noise with LB p_value "+ str(p_value)+ " \n")
        k_t = k_t - np.mean(k_t) # adjusting so that k_t sums to 0
        k_t = np.reshape(k_t,(1,len(k_t)))
        k_ts[i,:] = k_t
            
        #regional_re_est = regional_re_est + b_x@k_t
   
    # re-estimation of k_t terms
    res = 0
    if re_est:
        
        k_ts_reest = k_ts.flatten()
        print("Objecive before minimisation " +str(objective(k_ts_reest,a_xi, b_xs,D, region_pop, region_deaths,regional_terms)))
        result = minimize(objective, k_ts_reest, args=(a_xi, b_xs,D, region_pop, region_deaths,regional_terms), method='BFGS')
        print("Objective after minimisation " + str(objective(result.x,a_xi, b_xs,D, region_pop, region_deaths,regional_terms)))
        k_ts = result.x.reshape(((regional_terms,region_pop.shape[1])))
        
    #regional_re_est = regional_re_est + a_xi
    regional_re_est = a_xi + b_xs@np.diag(D)[:regional_terms,:regional_terms]@k_ts 
    #a_xi_adjustment = np.subtract(np.array(regional_logmort),regional_re_est)
    #a_xi = a_xi + np.mean(a_xi_adjustment,axis = 1).transpose()
    #regional_re_est = a_xi + B_xs@np.diag(D_country)[:common_terms,:common_terms]@K_ts + b_xs@np.diag(D)[:regional_terms,:regional_terms]@k_ts
    a_xi = np.reshape(np.mean(np.subtract(np.array(regional_logmort),regional_re_est),axis = 1),(regional_logmort.shape[0],1)) +a_xi
    regional_re_est = a_xi  + b_xs@np.diag(D)[:regional_terms,:regional_terms]@k_ts


        # models for regional terms 
    k_t_models = {}
    k_t_preds = np.zeros((regional_terms,train_period + prediction_period))
    k_t_preds_upconf = np.zeros((regional_terms,train_period + prediction_period))
    k_t_preds_lowconf = np.zeros((regional_terms,train_period + prediction_period)) #regional_logmort.shape[0]
    for i in range(regional_terms):
        k_t_model = pm.auto_arima(k_ts[i,:].flatten(),
                               exogenous_train,
                               start_p = 0,
                               start_q = 0,
                               max_p = 15,
                               max_q = 15,
                               max_d = 4,
                               m = 12,
                               seasonal = True,
                               suppress_warnings=True,
                               error_action='ignore',with_intercept = True)

        k_t_models[f'model_{i}'] = k_t_model
        k_t_model_preds,k_t_model_confint = k_t_model.predict(n_periods = prediction_period,X = exogenous_test,return_conf_int = True)
        k_t_preds[i,:] = np.append(k_ts[i,:],np.array(k_t_model_preds).reshape((1,len(k_t_model_preds))))
        k_t_preds_upconf[i,:] = np.append(k_ts[i,:],np.array(k_t_model_confint[:,0]).reshape((1,len(k_t_model_confint[:,0]))))
        k_t_preds_lowconf[i,:] = np.append(k_ts[i,:],np.array(k_t_model_confint[:,1]).reshape((1,len(k_t_model_confint[:,1]))))
            
            
        
    
    regional_predictions = a_xi  + b_xs@np.diag(D)[:regional_terms,:regional_terms]@k_t_preds  
    regional_predictions_upconf = a_xi +  b_xs@np.diag(D)[:regional_terms,:regional_terms]@k_t_preds_upconf 
    regional_predictions_lowconf = a_xi + b_xs@np.diag(D)[:regional_terms,:regional_terms]@k_t_preds_lowconf
    
    
    regional_preds = pd.DataFrame(regional_predictions)
    regional_preds.index = regional_data.index
    regional_preds.columns = regional_data.columns
    errors = MAPE(regional_preds.iloc[:,train_period:],regional_data.iloc[:,train_period:])
    model_error = np.sum(errors)
    
    prop_k_t_pos = k_ts
    prop_k_t_pos[prop_k_t_pos >0] = 1
    prop_k_t_pos[prop_k_t_pos <0] = 0
    prop_k_t_pos = np.sum(prop_k_t_pos,axis = 1)/prop_k_t_pos.shape[1]
    print("Proportion of k_t values which are positive" + str(prop_k_t_pos) + " \n")
    
    prop_b_x_pos = b_xs
    prop_b_x_pos[prop_b_x_pos > 0] = 1
    prop_b_x_pos[prop_b_x_pos < 0] = 0
    prop_b_x_pos = np.sum(prop_b_x_pos,axis = 0)/prop_b_x_pos.shape[0]
    print("Proportion of b_x values which are positive" + str(prop_b_x_pos) + " \n")
    
    for i in range(regional_terms):
        p_value = sm.stats.acorr_ljungbox(k_ts[i,:], lags=[len(k_ts[i,:]) - 1],return_df = True).iloc[0,1]
        print("p_value k_t" +str(i)+" after re-estimation: " +str(p_value)+ " \n")
        if p_value > 0.05:
                print("Regional k_t"+str(i)+ " is white noise with LB p_value "+ str(p_value)+" \n")
        

    return a_xi,b_xs,k_ts,regional_re_est,k_t_models,k_t_preds,regional_predictions,regional_predictions_upconf,regional_predictions_lowconf,model_error,res,D


In [1]:
## Functions for splitting data frame into epidemic years, 


def epi_year(df):
    epi_years = {}
    length = df.shape[1]
    years = int(length/12)
    for i in range(years):
        epi_years[f"year_{i}"] = df.iloc[:,(i*12 + 6):((i+1)*12 + 6)]
    return epi_years


def excess_sum(death_table_df):
        epi_years = epi_year(death_table_df)
        epi_year_avgs = {}
        print("Note: year "+ str(len(epi_years)-1) + " is only half an epiyear!  + First 6 months disregarded")
        for i in range(len(epi_years)):
            epi_year_avgs[f"year_{i}"] = epi_years[f"year_{i}"].sum(axis = 1)
            
        return epi_year_avgs
    
    
def epi_year_lin_ests(death_table_df):
    excess_avg = excess_sum(death_table_df)

    expected_death = np.zeros((death_table_df.shape[0],1))
    for i in range(len(excess_avg)-1):
        expected_death = expected_death + np.reshape(excess_avg[f"year_{i}"],expected_death.shape)

    expected_death = expected_death/(len(excess_avg)-1)
    return expected_death


#Averaging estimate for second half of epidemic year 
def est_half2_epiyear(death_table):
    death_count = {}
    expected_death = np.zeros((death_table.shape[0],))
    for i in range(len(epi_year(pd.DataFrame(death_table)).keys()) -1):
        death_count[f"year_{i}"] = epi_year(pd.DataFrame(death_table))[list(epi_year(pd.DataFrame(death_table)).keys())[i]].iloc[:,6:].sum(axis = 1)
        expected_death = expected_death + death_count[f"year_{i}"]

    expected_death = expected_death/len(death_count.keys())
    return expected_death

#Averaging estimate for first half of epidemic year
def est_half1_epiyear(death_table):
    death_count = {}
    expected_death = np.zeros((death_table.shape[0],))
    for i in range(len(epi_year(pd.DataFrame(death_table)).keys()) -1):
        death_count[f"year_{i}"] = epi_year(pd.DataFrame(death_table))[list(epi_year(pd.DataFrame(death_table)).keys())[i]].iloc[:,:6].sum(axis = 1)
        expected_death = expected_death + death_count[f"year_{i}"]

    expected_death = expected_death/len(death_count.keys())
    return expected_death


In [7]:
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()
# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")

print("Run Complete: "+ dt_string)

Run Complete: 28/03/2024 13:38:24
